In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import logging

from fundus_data_toolkit.datamodules import CLASSIF_PATHS, SEG_PATHS, USER_SETTING, Task, register_paths
from fundus_data_toolkit.datamodules.classification import (
    AptosDataModule,
    DDRDataModule,
    EyePACSDataModule,
    IDRiDDataModule,
)
from fundus_data_toolkit.datamodules.utils import merge_existing_datamodules
from nntools import NNOpt

root = logging.getLogger()
root.setLevel(logging.NOTSET)


In [8]:
paths = {
'EYEPACS': '/home/clement/Documents/data/eyepacs/',
'APTOS': '/home/clement/Documents/data/aptos/',
'DDR': '/home/clement/Documents/data/DDR/DDR-dataset/DR_grading/',
'IDRID': '/home/clement/Documents/data/IDRID/B. Disease Grading/', 
}
register_paths(paths, task=Task.CLASSIFICATION)

{'classification_eyepacs': '/home/clement/Documents/data/eyepacs/', 'classification_aptos': '/home/clement/Documents/data/aptos/', 'classification_ddr': '/home/clement/Documents/data/DDR/DDR-dataset/DR_grading/', 'classification_idrid': '/home/clement/Documents/data/IDRID/B. Disease Grading/'}


In [9]:
img_size = (512, 512)
idrid_datamodule = IDRiDDataModule(CLASSIF_PATHS.IDRID, img_size=img_size, batch_size=64, 
                                   valid_size=0.2,
                                   use_cache=True, num_workers=8,
                                   precise_autocrop=True,
                                   cache_option=NNOpt.CACHE_MEMORY).setup_all()
eyepacs_datamodule = EyePACSDataModule(CLASSIF_PATHS.EYEPACS, img_size=img_size, batch_size=64, use_cache=True, num_workers=8,
                                       precise_autocrop=True,
                                   cache_option=NNOpt.CACHE_MEMORY).setup_all()
aptos_datamodule = AptosDataModule(CLASSIF_PATHS.APTOS, img_size=img_size, batch_size=64, use_cache=True, num_workers=8,
                                   precise_autocrop=True,
                                   cache_option=NNOpt.CACHE_MEMORY).setup_all()
ddr_datamodule = DDRDataModule(CLASSIF_PATHS.DDR, img_size=img_size, batch_size=64, use_cache=True, num_workers=8,
                                   cache_option=NNOpt.CACHE_MEMORY, precise_autocrop=True).setup_all()
# ddr_datamodule.train.plot(0)